In [80]:
# libraries
import json
import lzma
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from IPython.core.display import display, HTML
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords  
import nltk
nltk.download('stopwords')
import matplotlib.pyplot as plt
import seaborn as sns
import random
sns.set()

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# pd.options.display.max_columns = 999
# pd.options.display.max_rows = 999

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qilongxin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# setting up tokenizer
tokenizer = RegexpTokenizer('\s+', gaps=True)

In [53]:
def get_raw_data(state):
    # reading json files
    cases = []
    with lzma.open(state + '/data/data.jsonl.xz', 'r') as jsonl_file:
        for case in jsonl_file:
            cases.append(json.loads(str(case, 'utf-8')))

    df = pd.DataFrame(cases).sort_values('decision_date').reset_index(drop=True)
    df['decision_date'] = pd.to_datetime(df['decision_date'])

    # parsing data
    storage = []
    for i in range(df.shape[0]):
        casebody_idx = df.columns.get_loc("casebody")
        judges = df.iloc[i,casebody_idx]['data']['judges']
        attorneys = df.iloc[i,casebody_idx]['data']['attorneys']
        headnotes = df.iloc[i,casebody_idx]['data']['head_matter']
        if df.iloc[i,casebody_idx]['data']['opinions'] != []:
            opinions = df.iloc[i,casebody_idx]['data']['opinions'][0]['text']

        storage.append({'judges': judges,
                        'attorneys': attorneys,
                        'headnotes': headnotes,
                        'opinions': opinions})
    df_parsed = pd.DataFrame(storage)
    df = df_parsed.merge(df, left_index=True, right_index=True)

    return df

In [54]:
df_ar = get_raw_data('Arkansas')

baseline_size = 10

baseline_data = df_ar.sample(n=baseline_size,replace=False, random_state=1)
baseline_text,baseline_summary = baseline_data.opinions.tolist(),baseline_data.headnotes.tolist()



In [82]:
from summarizer import Summarizer
#https://github.com/dmmiller612/bert-extractive-summarizer
import rouge 

model = Summarizer()
rouge = rouge.Rouge()

def getRougeScore(y_true,y_pred,metric='rouge-2',score='f'): 
    scores = np.zeros(len(y_true))
    for i,text in enumerate(y_true): 
        full = ''.join(result)
        sc = rouge.get_scores(baseline_summary[i], full)
        scores[i] = sc[0][metric][score]
    print(np.mean(scores))
    return scores 

class Baseline_BERT_model(): 
    def __init__ (self,min_length=60): 
        super(Baseline_BERT_model,self).__init__()
        self.min_length = min_length
        
    def evaluate(self,texts,summaries):  
        results = []
        for i,text in enumerate(texts): 
            results.append(model(text,min_length=self.min_length))
        self.results = results
        self.summaries = summaries 
        return results 
    
    def reportEvaluation(self,metric='rouge-2',score='f'): 
        _ = getRougeScore(self.summaries,self.results,metric,score) 
    def inspectOneSummary(self):
        # randomly inspect one summary 
        print(self.results[random.randint(0,len(self.results)-1)])
        

In [83]:
bert = Baseline_BERT_model()

_ = bert.evaluate(baseline_text,baseline_summary)
bert.reportEvaluation()


0.01252517567017948


In [84]:
bert.inspectOneSummary()

Appellant, Wayne Burrow, was charged with refusing to send a minor to school, in violation of Ark. Stat. 1980):
Every parent, guardian, or other person residing within the State of Arkansas and having in custody or charge any child or children between the ages of seven [7] and fifteen [15], (both inclusive) shall send such child or children to a public, private, or parochial school under such penalty for noncompliance with this section as hereinafter provided. Appellant urges us to find the statute unconstitutionally vague. He argues that our law requires attendance of children at a “public, private or parochial school” but fails to give any definition of what constitues school and, therefore, does not provide fair notice of the conduct that is prohibited. If, by his action, that individual clearly falls within the conduct proscribed by the statute he cannot be heard to complain. We stated the traditional standard for determining vagueness in Jordan v. State, 274 Ark. 572, 626 S.W.2d 9